In [1]:
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
trainimggen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                  horizontal_flip=True, 
                                  rotation_range=30, 
                                  width_shift_range=3, 
                                  height_shift_range=3,
                                  brightness_range=None,
                                  shear_range=3,
                                  zoom_range=3,
                                  channel_shift_range=3)
validationimggen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [3]:
batch_size=16
filepath_train = '../data/Expert_TrainEval/Train'
filepath_val = '../data/Expert_TrainEval/Val'
train_small = trainimggen.flow_from_directory(filepath_train, target_size=(100,100), batch_size=batch_size)
val_small = validationimggen.flow_from_directory(filepath_val, target_size=(100,100), batch_size=batch_size)

Found 534 images belonging to 3 classes.
Found 99 images belonging to 3 classes.


In [4]:
from tensorflow.keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

nb_filters = 32
kernel_size = (3, 3)
pool_size = (2, 2)
input_size = (100,100,3)
n_categories=3

model = Sequential()
# 2 convolutional layers followed by a pooling layer followed by dropout
model.add(Convolution2D(nb_filters, kernel_size,
                        padding='valid',
                        input_shape=input_size))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
# transition to an mlp
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_categories))
model.add(Activation('softmax'))

In [5]:
model.compile(optimizer='SGD', 
              loss=['categorical_crossentropy'], 
              metrics=['accuracy'])

In [14]:
training_images = 32
steps_per_epoch = training_images // batch_size
validation_steps = training_images // batch_size
steps_per_epoch, validation_steps


(2, 2)

In [17]:
model.fit_generator(generator=train_small, 
            epochs=10, 
            steps_per_epoch=25, 
            validation_steps=10)

TypeError: fit_generator() got an unexpected keyword argument 'n_jobs'

In [12]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model

def create_transfer_model(input_size, n_categories, weights = 'imagenet'):
        # note that the "top" is not included in the weights below
        base_model = Xception(weights=weights,
                          include_top=False,
                          input_shape=input_size)
        
        model = base_model.output
        model = GlobalAveragePooling2D()(model)
        predictions = Dense(n_categories, activation='softmax')(model)
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model
    
transfer_model = create_transfer_model(input_size=(100,100,3), n_categories=3)

AttributeError: module 'h5py' has no attribute 'File'

In [23]:
import tensorflow.python.keras,os
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [29]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="../data/Expert_TrainEval/Train",target_size=(224,224))
vldata = ImageDataGenerator()
valdata = vldata.flow_from_directory(directory="../data/Expert_TrainEval/Val", target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="../data/Expert_TrainEval/Test", target_size=(224,224))

Found 534 images belonging to 3 classes.
Found 99 images belonging to 3 classes.
Found 36 images belonging to 3 classes.


In [41]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3, activation="softmax"))

In [42]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=tensorflow.keras.losses.categorical_crossentropy, metrics=['accuracy'])

/Users/mbun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [43]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 56, 56, 256)      

In [45]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=10,generator=traindata, validation_data=valdata, validation_steps=5,epochs=10,callbacks=[checkpoint,early])

/Users/mbun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/100
100/100 [==============================] - 215s 2s/step - loss: 1008.0385 - accuracy: 0.3390 - val_loss: 1.1447 - val_accuracy: 0.3333
